In [1]:
import pandas as pd
import re
from tqdm.notebook import tqdm

def get_second(time_str):
    segment = time_str.split(' ')
    second = 0    
    for i in segment:
        if i[-1] == 's':
            second += int(i[:-1])
        elif i[-1] == 'm':
            second += int(i[:-1]) * 60
        elif i[-1] == 'h':
            second += int(i[:-1]) * 60 * 60
        elif i[-1] == 'd':
            second += int(i[:-1]) * 60 * 60 * 24
        else:
            print(time_str)
    return second

df = pd.read_csv('../dataset/raw_dataset.csv',parse_dates=['created_at'])
df['is_bot'] = df['is_bot'].astype(bool)
print(df.dtypes)

id                          int64
change_id                  object
message_id                 object
author                      int64
real_author                 int64
created_at         datetime64[ns]
message                    object
revision_number             int64
is_bot                       bool
username                   object
dtype: object


In [2]:
## create boxplot for additional cost

# Recheck [waiting time / code review time / computation time / propotion (waiting time in code review time)]
# Overall [waiting time / code review time / computation time]

tqdm.pandas()
output = dict()

df.sort_values(by=['created_at'], inplace=True)
for key, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    change_id , revision_number = key
    build, recheck, build_then_recheck = False, False, False 
    final_status = dict()
    prev_status = dict()
    computation_time = 0
    
    total_max_time = 0
    max_time = 0

    dict_bot = {}
    rnd = 0
    
    start_time = ''
    end_time = ''

    recheck_count = 0
    
    for ind,col in group.iterrows():
        
        time = col['created_at']
        
        if not start_time:
            start_time = time
            
        if not end_time or time > end_time:
            end_time = time
        
        #start loop if build fail is found
        if not build and col['is_bot'] and 'build' in col['message'].lower():
            build = True
            
        if not build:
            continue
                                                
        if col['is_bot']:
            try:
                patch_fail = False
                build = re.findall(r'((?:merge|build) (?:succeeded|failed))',col['message'],re.IGNORECASE)
                bot = re.findall(r'((?:success|failure|aborted) in(?: \w+?)*s)',col['message'],re.IGNORECASE)
                
                if any('fail' in b.lower() for b in build): #or any('fail' in b.lower() for b in bot):
                    patch_fail = True
                elif any('abort' in b.lower() for b in build): #or any('abort' in b.lower() for b in bot):
                    patch_fail = True
                
                if build and recheck:                
                    bot_time = 0
                    for i in bot:
                        bot_time += get_second(i.split(' in ')[1])
                        
                        if max_time < get_second(i.split(' in ')[1]):
                            max_time = get_second(i.split(' in ')[1])
                        
                    computation_time += bot_time
            except:
                pass
  
            if rnd not in dict_bot:
                dict_bot[rnd] = dict()

            if col['username'] not in dict_bot[rnd]:
                dict_bot[rnd][col['username']] = ''

            
            if dict_bot[rnd][col['username']] == 'failure':
                final_status[col['username']] = 'failure'
            else:
                
                if patch_fail:
                    dict_bot[rnd][col['username']] = 'failure'
                else:
                    dict_bot[rnd][col['username']] = 'success'

                final_status[col['username']] = 'failure' if patch_fail else 'success'


        
        if 'recheck' in col['message'].lower() and col['is_bot'] == False:
            recheck = True
            rnd += 1
            
            if len(final_status) > 0:
                prev_status = final_status
            final_status = dict()
            
            total_max_time += max_time
            max_time = 0
            
            if build:
                build_then_recheck = True
            
            recheck_count += 1
            
    total_max_time += max_time
    if build_then_recheck:
        
        status = ''
        
        if len(final_status) == 0:
            final_status = prev_status
        
        for username in final_status:
            if status == 'failure':
                continue
            if final_status[username] == 'failure':
                status = 'failure'
            if final_status[username] == '':
                status = ''
            if final_status[username] == 'success':
                status = 'success'
        
        output[key] = {'computation_time': computation_time,'status': status, 'bot':dict_bot, 'waiting_time':total_max_time, 'code_review_time': (end_time - start_time).total_seconds(), 'recheck_count': recheck_count}
        

  0%|          | 0/267239 [00:00<?, ?it/s]

In [3]:
output_tup = list()

dropped_case, added_case, replaced_case = 0,0,0
same_case_1, dropped_case_1, added_case_1, replaced_case_1 = 0,0,0,0

for key in tqdm(output):
    
#     proportion = round(output[key]['waiting_time'] * 100 / output[key]['code_review_time'] ,2)

    computation_time = output[key]['computation_time']
    waiting_time = output[key]['waiting_time']
    code_review_time = output[key]['code_review_time']
    status = output[key]['status']
    
    bots = output[key]['bot']
    first_bot = bots[0]
    last_bot = bots[list(bots.keys())[-1]]
    
    
    if len(bots) >= 100:
        print(key, len(bots))
    
    # scenario 
    
    # scenario #1 [Changed outcome] Justifiable
    # scenario #2 [Changed outcome] Repeated builds
    # scenario #3 [No Change] Single recheck
    # scenario #4 [No Change] Multiple recheck

    if status == 'success' and len(bots) == 2:
        scenario = 1
    elif status == 'success' and len(bots) != 2:
        scenario = 2
    elif status == 'failure' and len(bots) == 2:
        scenario = 3
    elif status == 'failure' and len(bots) != 2:
        scenario = 4
    else:
        print("should not be here", key)
    
    bot_modify = 0
    
    # l_key union f_key
    f_u_l = set(first_bot.keys()).union(set(last_bot.keys()))
    # cehck if L == F
    if set(first_bot.keys()) == set(last_bot.keys()):
        bot_modify = 'same bot'
    # check if L in F, FuL == F [dropped case]
    elif set(first_bot.keys()) == f_u_l:
        bot_modify = 'drop bot'
    # check if F in L, FuL == L [added case]
    elif set(last_bot.keys()) == f_u_l:
        bot_modify = 'add bot'
    # replace case
    else:
        bot_modify = 'replace bot'
    
    output_tup.append((scenario, bot_modify, status, computation_time, waiting_time, code_review_time))
recheck_df = pd.DataFrame(output_tup, columns=['scenario', 'bot_modify', 'status', 'computation_time', 'waiting_time', 'code_review_time'])

  0%|          | 0/36494 [00:00<?, ?it/s]

('I9a5b48d58ff9a1ba651468b818cc669e1ca099ff', 3) 118
('Ibea13629fafc40ffd9ba64a318dcfd691c88a1dc', 1) 175


In [4]:
# Proportion of Patch Sets with Changed and Unchanged CI Outcomes After Rechecks

print(recheck_df['scenario'].value_counts())
print(recheck_df['scenario'].value_counts(normalize=True))

3    11884
1     9795
4     8282
2     6533
Name: scenario, dtype: int64
3    0.325643
1    0.268400
4    0.226941
2    0.179016
Name: scenario, dtype: float64
